# TP 2-3 : Branch-and-bound applied to a knapsack problem

### Initialisation (à faire une seule fois)

In [ ]:
import Pkg; 
Pkg.add("GraphRecipes"); Pkg.add("Plots"); 
using GraphRecipes, Plots #only used to visualize the search tree at the end of the branch-and-bound

### Récupération des données

In [3]:
function readKnaptxtInstance(filename)
    price=[]
    weight=[]
    KnapCap=[]
    open(filename) do f
        for i in 1:3
            tok = split(readline(f))
            if(tok[1] == "ListPrices=")
                for i in 2:(length(tok)-1)
                    push!(price,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "ListWeights=")
                for i in 2:(length(tok)-1)
                    push!(weight,parse(Int64, tok[i]))
                end
            elseif(tok[1] == "Capacity=")
                push!(KnapCap, parse(Int64, tok[2]))
            else
                println("Unknown read :", tok)
            end 
        end
    end
    capacity=KnapCap[1]
    return price, weight, capacity
end

readKnaptxtInstance (generic function with 1 method)

### Tests de sondabilités TA, TO et TR basés sur le modèle linéaire

In [4]:
function TestsSondabilite(noeud, BestProfit)

    TA, TO, TR = false, false, false
    # Test d'admissibilité : vérifié si la capacité restante est négative
    if(noeud.capacite_restante <= 0)
        TA=true
        return TA, TO, TR

    else 
        # Test d'optimalité
        if(noeud.borne_sup <= BestProfit) 
            TO=true
        end
        # Test de résolution 1 : réussit si les variables de décisions sont entières
        if(abs(noeud.borne_sup - noeud.borne_inf) <= 10^-5)
            TR=true
        end 
    end 

    return TA, TO, TR

end

TestsSondabilite (generic function with 1 method)

### Procédure de séparation (branching) et stratégie d'exploration permettant de se placer au prochain noeud à traiter

In [5]:
"""
Fonction pour créer le noeud fils gauche d'un noeud non-sondable 
Précondition : Le noeud est non-sondable 
"""
function SeparerNoeud!(listobjs, listvals, nb_var,ratio)

    ratio_dispo = zeros(nb_var);

    # Vérification des variables de décisions déjà fixées 
    for indice in 1:nb_var
        # Si la variable xi n'est pas encore fixée alors elle est disponible
        if(!(indice in listobjs))
            ratio_dispo[indice] = ratio[indice]
        end
    end

    ratio_max, indice_max = findmax(ratio_dispo)
    
    # Création du noeud fils gauche avec xi = 1
    push!(listobjs,indice_max)
    push!(listvals,1.0) 
end

SeparerNoeud!

In [6]:
"""
Fonction pour explorer un autre noeud lorsque le noeud actuel est sondable
Précondition : le noeud actuel est sondable
Renvoie : 
    - True : Si on explore à partir d'un noeud racine sondable
             OU Si l'algo a fini de construire l'arbre
    - False : Si l'algo a crée un noeud fils droit 

-----------------------------------------------------------------------------
ALGORITHME : 

Si OnEstPasSurNoeudRacine Alors
    OnRetourneAuNoeudParent
        
    TantQue OnEstRemonteParFilsDroit && OnEstSurNoeudPasRacine
        OnRemonte
    FinTantQue

    Si OnEstRemonteParNoeudFilsGauche Alors 
        OnCreeLeNoeudFilsDroit
        False
    Sinon  (On se situe sur le noeud racine => on a fini de construire l'arbre)
        True
Sinon 
    True
-----------------------------------------------------------------------------
"""

function ExplorerAutreNoeud!(listobjs, listvals, listnodes)
    #this node is sondable, go back to parent node then right child if possible
    
    stop=false
    # On se situe sur un noeud qui n'est pas le noeud racine (0)
    if (length(listobjs)>= 1)
        # Retour au noeud parent
        obj=pop!(listobjs)
        theval=pop!(listvals)
        tmp=pop!(listnodes)

        #go to right child if possible, otherwise go back to parent
        # Sur la branche remontée, la valeur de xi = 0, noeud parent != racine
        # Remonte jusqu'au noeud où la branche parent xi = 1 OU noeud racine
        while( (theval==0.0) && (length(listobjs)>= 1))
            obj=pop!(listobjs)
            theval=pop!(listvals)
            tmp=pop!(listnodes)
        end

        # Sur la branche remontée, la valeur de xi = 1 
        if theval==1.0
            # On crée le noeud fils droit avec xi = 0
            push!(listobjs,obj)
            push!(listvals,0.0)
        # On est remonté par que des branches xi = 0 jusqu'au noeud racine
        # = on a fini d'explorer/de construire l'arbre du problème
        else
            #println("\nFINISHED")
            stop=true
        end

    # On se situe sur un noeud qui est sur le noeud racine (0)
    else
        #the root node was sondable
        #println("\nFINISHED")
        stop=true
    end
    return stop 
end

ExplorerAutreNoeud! (generic function with 1 method)

### Calcul de la borne

In [7]:
function CalculBorne(listobjs, listvals, capacite_initiale, prix, poids, option)

    # Calcul du ratio des objets 
    ratio = prix ./ poids
    # Tri ratio par ordre croissant (pour dépiler dans l'ordre décroissant)
    indice_croissant = sortperm(ratio)
    ratio_croissant = ratio[indice_croissant]
    # Initialisation
    borne_inf = 0
    borne_sup = 0
    capacite_restante = capacite_initiale
    k = 0   # indice de parcours

    # Calcul de la borne inf
    # On recherche les variables de décisions (les indices) déjà fixées 
    for indice in listobjs
        k += 1
        capacite_restante -= listvals[k] * poids[indice]
        borne_inf += listvals[k]*prix[indice]
        # On dépile les variables utilisées/fixées
        pop!(indice_croissant)
        pop!(ratio_croissant)
    end

    # Calcul de la borne sup
    borne_sup = borne_inf
    # Calcul Borne1
    if option == 1 && (capacite_restante > 0)
        if (ratio_croissant == [])
            ratio_max = 0
        else
            ratio_max = pop!(ratio_croissant)
        end     
        borne_sup += capacite_restante * ratio_max

    # Calcul Borne2
    elseif option == 2
        capacite = capacite_restante
        # Tant qu'on a pas rempli le sac OU qu'il n'y plus d'objet à mettre
        while (capacite > 0) && (indice_croissant != [])
            indice_dominant = pop!(indice_croissant)
            ratio_max = pop!(ratio_croissant)
            # On peut mettre l'objet suivant entièrement
            if poids[indice_dominant] <= capacite
                capacite -= poids[indice_dominant]
                borne_sup +=  prix[indice_dominant]
            # Sinon on ajoute une fraction de l'objet
            else
                borne_sup += capacite * ratio_max
                capacite = 0
            end 
        end
    elseif (option != 0) && (option != 1)
        println("ERREUR option");
    end


    return borne_inf, borne_sup, capacite_restante

end

CalculBorne (generic function with 1 method)

### Boucle principale : résoudre une relaxation, appliquer les tests de sondabilité, identifier le prochain noeud, répéter.

In [8]:
mutable struct Noeud
    numero::Int
    capacite_restante::Int
    borne_sup::Float64
    borne_inf::Float64
end

"""
Fonction pour mettre à jour éventuellement une solution réalisable (ou optimale)
"""
function MAJ_Solution!(
    noeud::Noeud, listobjs::Vector, listvals::Vector, BestProfit,
    BestSol::Vector, TO::Bool, TA::Bool)
    # Si le TO ou TA est vérifié, alors on ne met pas à jour la meilleure solution
    if (TO || TA)
        
    # Si TO n'est pas vérifié, on regarde si on peut mettre à jour la solution
    # On a trouvé une meilleure solution réalisable 
    elseif (noeud.borne_inf  >= BestProfit)
        # On met à jour le maximum des bénéfices 
        BestProfit = noeud.borne_inf 
        k = 1
        BestSol = zeros(length(BestSol))

        # On met à jour la solution x_sol
        for indice in listobjs 
            BestSol[indice] = listvals[k]
            k += 1
        end
    end
    
    return BestProfit, BestSol
end

MAJ_Solution!

In [9]:
mutable struct Noeud
    numero::Int
    capacite_restante::Int
    borne_sup::Float64
    borne_inf::Float64
end

# option : choix du calcul de la borne
#          option == 1 -> Borne 1
#          option == 2 -> Borne 2
function main(afficher::Bool,filename::String,option::Int)

    ## INITIALISATION DU PROBLEME
    prix, poids, capacite_initiale = readKnaptxtInstance(filename)
    nb_var = length(prix)

    #create the structure to memorize the search tree for visualization at the end
    trParentnodes=Int64[] #will store orig node of arc in search tree
    trChildnodes=Int64[] #will store destination node of arc in search tree
    trNamenodes=[] #will store names of nodes in search tree

    #intermediate structure to navigate in the search tree
    listobjs=[]
    listvals=[]
    listnodes=[]

    ratio = prix./poids
    BestProfit=-1
    BestSol=zeros(nb_var)

    current_node_number=0
    stop = false

    ## BOUCLE PRINCIPALE
    while(!stop)

        if afficher
            println("\n******\n\nNode number ", current_node_number, " : \n---------------")
        end

        ## Mise à jour de l'affichage graphique de l'arbre
        MAJ_Graphique(current_node_number, listnodes, trNamenodes, trParentnodes, trChildnodes)

        ## ETAPE 1 : Calcul des bornes
        borne_inf, borne_sup, capacite_restante = CalculBorne(listobjs, listvals, capacite_initiale, prix, poids, option)

        ## ETAPE 2  Construction du noeud 
        noeud = Noeud(current_node_number,capacite_restante,borne_sup,borne_inf)
        push!(listnodes, noeud)

        ## ETAPE 3 : Réaliser les tests de sondabilités
        TA, TO, TR = TestsSondabilite(noeud, BestProfit)
        noeud_sondable = TA || TO || TR        

        ## ETAPE 4 : Mise à jour éventuelle d'une solution réalisable (ou optimale)
        BestProfit, BestSol = MAJ_Solution!(noeud, listobjs, listvals, BestProfit, BestSol, TO,TA)

        if afficher
            AffichageResultats(BestProfit,BestSol,capacite_restante,borne_sup,borne_inf,TA,TO,TR)
        end
        
        ## ETAPE 5 : Séparation ou Exploration selon la sondabilité
        if(!noeud_sondable)
            SeparerNoeud!(listobjs, listvals, nb_var,ratio)
        else
            stop = ExplorerAutreNoeud!(listobjs, listvals, listnodes)
        end

        ## ETAPE 6 : Répéter la boucle si on n'a pas fini de construire l'arbre
        current_node_number = current_node_number + 1
    end

    if afficher
        println("\n******\nf_max = ", BestProfit, "\nx_max = ", BestSol,
                "\nNb_noeuds = ", length(trNamenodes))
    end

    return BestProfit, BestSol, trParentnodes, trChildnodes, trNamenodes

end 

main (generic function with 1 method)

### Affichage du résultat final

In [10]:
function AffichageResultats(BestProfit,BestSol,capacite_restante,borne_sup,borne_inf,TA,TO,TR)
    println("Current optimal value = ", BestProfit, "\nCurrent optimal x = ", BestSol)
    println("Capacité restante : Q = ", capacite_restante)
    println("Borne sup = ", borne_sup)
    println("Borne inf = ", borne_inf)
    println("Test d'admissibilité : TA = ", TA)
    println("Test d'optimalité : TO = ", TO)
    println("Test de résolution : TR = ", TR)
end 

AffichageResultats (generic function with 1 method)

In [11]:
mutable struct Noeud
    numero::Int
    capacite_restante::Int
    borne_sup::Float64
    borne_inf::Float64
end

"""
Fonction qui réalise la mise à jour de l'affichage graphique de l'arbre
"""
function MAJ_Graphique(current_node_number, listnodes, trNamenodes, trParentnodes, trChildnodes)
    push!(trNamenodes,current_node_number+1) 
    if(length(trNamenodes)>=2)
        push!(trParentnodes,listnodes[end].numero + 1) # +1 because the 1st node is "node 0"
        push!(trChildnodes, current_node_number+1) # +1 because the 1st node is "node 0"
    end
end 

MAJ_Graphique

In [ ]:
using GraphRecipes, Plots #only used to visualize the search tree at the end of the branch-and-bound
afficher_calcul = false
probleme_teste = "instancesETU/KNAPnewformat/test.opb.txt"
option = 2

BestProfit, Bestsol, trParentnodes, trChildnodes, trNamenodes = main(afficher_calcul,probleme_teste,option)


### Questions préliminaires : 

Q1) La règle de séparation de l'exemple choisit une variable dans l'ordre lexicographique de l'indice et divise le problème 
    en deux sous-problèmes avec xi = 1 et xi = 0

Q2) Dans cet exemple, la borne superieure est calculée avec la fonction objective_value(model2)

Q3) TA : utilisation de termination_status(model2) pour vérifier si la résolution est faisable ou pas. 
         Si elle n'est pas faisable le TA est vérifiée
    TO : Le test d'optimisalité est vérifié si la borne sup du noeud est inférieure ou égale au bénéfice de la meilleure solution enregistrée.
    TR : Le test de résolution est vérifié si la solution est entière

Q4) La stratégie d'exploration adoptée ici est une exploration en profondeur avec une priorité au noeud de gauche qui vient d'être
    avec xi = 1.
    Quand il n'est plus possible de créer un noeud à gauche, il retourne au noeud parent pour créer un noeud droit (xi = 0) 
    si c'est possible.

### Questions (Analyse et code)

Q1) Calcul borne 1 --> option == 1 // Calcul borne 2 --> option == 2  

Q2) OK  

Q3)  
Règle de séparation --> cf. SeparerNoeud!.jl  
     TA,TO,TR --> cf. TestsSondabilite.jl  
     Stratégie d'exploration --> cf. ExplorerAutreNoeud!.jl  

Q4) Pour la structure de données, nous avons besoin d'une structure de données dynamique puisque l'algorithme du branch and bound implémente de manière itérative de nouveaux noeuds. De même, lors de l'exploration d'un autre noeud, il est nécessaire de remonter au noeud parent pour créer de nouveaux fils. Enfin, pour chaque noeud, on doit calculer la quantité restante et les bornes. C'est pourquoi, nous avons défini un objet composite mutable nommé Noeud avec les attributs citées ci-dessus.  

Q5) Pour comparer les performances du Branch-and-Bound avec la borne 1 et la borne 2, nous nous sommes restreints aux jeux de données suivantes :  
1. almost_strongly_correlated 
2. circle 
3. inverse_strongly_correlated 
4. multiple_strongly_correlated  

En ce qui concerne les données testées, elles ont été importées grâce à la fonction ImporterDonnees.jl. La plus grande quantité de données testée a été limité à 200 variables. Au delà, l'exécution du programme ne terminait pas.  
Cela doit probablement s'expliquer par la qualité de notre code. En effet, nos algorithmes ont été codés sans prendre en considération leur complexité.  

Malgré tout, on peut tout de même tirer quelques conclusions de nos tests :  
1. On obtient les mêmes solutions (f_max, x_sol ?) pour la borne1 et la borne2
2. Avec le calcul de borne 2, l'algorithme est toujours plus performant en terme de nombre de noeuds crée. Ceci est cohérent dans la mesure on calcule une borne sup plus précise, donc plus petite que celle avec la borne1. Ainsi on augmente les chances qu'un noeud vérifie le test d'optimalité, et donc le nombre de noeuds crée.
3. Cependant, l'amélioration de la performance dépend grandement de la configuration des données. Par exemple, on peut remarquer dans le cas où les données sont fortement inversement correlées, les performances sont presque identiques. 

### Comparaison performance Borne1 et Borne2

In [19]:
include("src/ImporterDonnees.jl")
almost_strongly_correlated, circle, inverse_strongly_correlated, multiple_strongly_correlated = ImporterDonnees()
afficher_calcul = false
# Choisir parmi ceux qui sont importés
jeu_données = almost_strongly_correlated
borne = 2

for donnees in jeu_données
    # On récupère le nom du fichier des données analysées
    _,  filename = rsplit(donnees,"/";limit = 2)
    if borne == 1
        BestProfit, BestSol, trParentnodes, trChildnodes, trNamenodes = main(afficher_calcul,donnees,borne)
        println("\n**** NOM DU FICHIER AVEC BORNE 1 : $filename \nTaille du problème : nb_var = ", 
        length(BestSol), "\nf_max = ", BestProfit,
        "\nNb_noeuds = ", length(trNamenodes));
    elseif borne == 2
        BestProfit, BestSol, trParentnodes, trChildnodes, trNamenodes = main(afficher_calcul,donnees,borne)
        println("\n**** NOM DU FICHIER AVEC BORNE 2 : $filename \nTaille du problème : nb_var = ", 
        length(BestSol), "\nf_max = ", BestProfit,
        "\nNb_noeuds = ", length(trNamenodes));
    end
end



**** NOM DU FICHIER AVEC BORNE 2 : knapPI_5_50_1000_1_-2096.opb.txt 
Taille du problème : nb_var = 50
f_max = 2095.0
Nb_noeuds = 6757

**** NOM DU FICHIER AVEC BORNE 2 : knapPI_5_50_10000_1_-21980.opb.txt 
Taille du problème : nb_var = 50
f_max = 21980.0
Nb_noeuds = 329



**** NOM DU FICHIER AVEC BORNE 2 : knapPI_5_100_1000_1_-2295.opb.txt 
Taille du problème : nb_var = 100
f_max = 2295.0
Nb_noeuds = 2821

**** NOM DU FICHIER AVEC BORNE 2 : knapPI_5_100_10000_1_-23965.opb.txt 
Taille du problème : nb_var = 100
f_max = 23965.0
Nb_noeuds = 1653



**** NOM DU FICHIER AVEC BORNE 2 : knapPI_5_200_1000_1_-2706.opb.txt 
Taille du problème : nb_var = 200
f_max = 2706.0
Nb_noeuds = 2603

**** NOM DU FICHIER AVEC BORNE 2 : knapPI_5_200_10000_1_-28310.opb.txt 
Taille du problème : nb_var = 

200
f_max = 28310.0
Nb_noeuds = 316425



**** NOM DU FICHIER AVEC BORNE 2 : knapPI_5_500_1000_1_-7241.opb.txt 
Taille du problème : nb_var = 500
f_max = 7240.0
Nb_noeuds = 629367
